## Pull All Youtube Videos from a Playlist
#### @author: christiangrech

In [14]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import json
import urllib
import isodate
import datetime
import pandas as pd

keys_file = open("key.txt")
lines = keys_file.readlines()


DEVELOPER_KEY = lines[0].rstrip()
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
PLAYLIST_ID = "PL-ee8SSNXhJhpLTBjvugTNl2o99Zqyjxq" # Prime time
PLAYLIST_ID = "PL-ee8SSNXhJhn4CZphkPQY0D_rliJ5QHI"  # first four
PLAYLIST_ID = "PLzrpE0iRHx_qc6TCuBqW9JkJZH9q-uk8r" # Natura doc
PLAYLIST_ID = "PLKTZIA5MRAocmtEQep08g44sYJ_58u7j1" # Rallies politika
PLAYLIST_ID = "PLKTZIA5MRAoc70n1WDdQNJQ85xdTPKZyO" # politika
PLAYLIST_ID = "PLZGlYv8NpNKT9P3xraGf0_NuWgh92u0pE"  # stejjer
PLAYLIST_ID = "PLZGlYv8NpNKQDngcd6w6n1_vpsU5BFBKL" # cartoons
PLAYLIST_ID = "PLZGlYv8NpNKTwXWCJ3wjsgtd-r0yj8PeK" # rakkonti
PLAYLIST_ID = "PLZGlYv8NpNKSdB6buG7wWbzptdTU5plzT" # stejjer 4 sena
PLAYLIST_ID = "PLZGlYv8NpNKRKIzdrHuocVMeZlFSOycP0" # stejjer 3 sena
PLAYLIST_ID = "PLZGlYv8NpNKSjSdvQxNUeoPrkHcRAfWx5" # stejjer 2 sena
PLAYLIST_ID = "PLZGlYv8NpNKT9P3xraGf0_NuWgh92u0pE" # stejjer 1 sena

PLAYLIST_ID = "PLZGlYv8NpNKSRbqwdo3iffmjfIrOE9M6j" # stejjer 0 sena
PLAYLIST_ID = "PLZGlYv8NpNKQO9OtCRPKO-6Xowzk6P1vU" # stejjer 


In [15]:
def fetch_all_youtube_videos(playlistId):
    """
    Fetches a playlist of videos from youtube
    We splice the results together in no particular order

    Parameters:
        parm1 - (string) playlistId
    Returns:
        playListItem Dict
    """
    youtube = build(YOUTUBE_API_SERVICE_NAME,
                    YOUTUBE_API_VERSION,
                    developerKey=DEVELOPER_KEY)
    res = youtube.playlistItems().list(
    part="contentDetails",
    playlistId=playlistId,
    maxResults="50"
    ).execute()

    nextPageToken = res.get('nextPageToken')
    while ('nextPageToken' in res):
        nextPage = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=playlistId,
        maxResults="50",
        pageToken=nextPageToken
        ).execute()
        res['items'] = res['items'] + nextPage['items']

        if 'nextPageToken' not in nextPage:
            res.pop('nextPageToken', None)
        else:
            nextPageToken = nextPage['nextPageToken']

    return res

In [16]:
if __name__ == '__main__':
    
  # comedy central playlist, has 332 video
  # https://www.youtube.com/watch?v=tJDLdxYKh3k&list=PLD7nPL1U-R5rDpeH95XsK0qwJHLTS3tNT
    info = {}
    info['links'] = []
    info['duration'] = []
    total_dur = datetime.timedelta(0)
    videos = fetch_all_youtube_videos(PLAYLIST_ID)
    for video in videos['items']:
        video_id = video['contentDetails']['videoId']
        url = f'https://www.youtube.com/watch?v={video_id}'
        search_url = f'https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={DEVELOPER_KEY}&part=contentDetails'
        req = urllib.request.Request(search_url)
        response = urllib.request.urlopen(req).read().decode('utf-8')
        data = json.loads(response)
        all_data = data['items']
        info['links'].append(url)
        dur= all_data[0]['contentDetails']['duration']
        duration = isodate.parse_duration(dur)
        total_dur = total_dur + duration
        info['duration'].append(str(duration))
    


In [17]:
df = pd.DataFrame(data=info)
df

,links,duration
0,https://www.youtube.com/watch?v=v_8bs0zuLS8,0:16:11
1,https://www.youtube.com/watch?v=7-RWj4KTI4c,0:11:24
2,https://www.youtube.com/watch?v=HFKYH3iUtq0,0:13:13
3,https://www.youtube.com/watch?v=xLM8vqlINp0,0:14:45
4,https://www.youtube.com/watch?v=XwDFCDc2d-U,0:14:07
...,...,...
78,https://www.youtube.com/watch?v=p9Hr5OE9TKg,0:16:53
79,https://www.youtube.com/watch?v=6cNQNzr5LGQ,0:13:31
80,https://www.youtube.com/watch?v=WVrRvfPC08Y,0:12:14
81,https://www.youtube.com/watch?v=SiRMtWftlGM,0:15:51


In [19]:
df.to_csv('/Users/christiangrech/Documents/GitHub/Youtube-Playlist-Info-Extractor/dict1.csv')

In [20]:
print(total_dur)

18:01:31
